# PicoRV32 Processor Mixed-Memory Processor Demo

This notebook demonstrates using Jupyter Notebooks and IPython Magics to run C/C++ and assembly code on a PicoRV32 Processor. 

The PicoRV32 Processor in this example is a Mixed-Memory processor. This means, it has a 64-KB BRAM Memory space, **AND** a 256 MB DDR Memory space (mapped at address 0x1000_0000). 

When arguments are passed to the PicoRV32 processor for execution they will be copied and passed in **DDR Memory** as PYNQ Contiguous Memory Allocated (CMA) arrays. Previously allocated CMA arrays passed as arguments will not be copied, and reused.

When the program terminates, results are back-propogated (if necessary). 

## Loading the Overlay

To begin, import the overlay using the following cell. This also loads the IPython Magics: `riscvc`, `riscvcpp`, and `riscvasm`. 

In [6]:
import pynq
print(pynq.__version__)

2.6.2


In [3]:
from pynq import Overlay

In [1]:
from riscvonpynq.picorv32.axi.picorv32 import Overlay
overlay = Overlay("/home/xilinx/jupyter_notebooks/GPCP/designs/picorv32/axi/picorv32_wrapper.bit")

In [7]:
overlay.pr_download("cp_F_const", "/home/xilinx/jupyter_notebooks/GPCP/designs/picorv32/axi/cp_F_const_cp_0_const_partial.bit")

AttributeError: Could not find IP or hierarchy cp_F_const in overlay

In [4]:
overlay.set_partial_region

AttributeError: Could not find IP or hierarchy set_partial_region in overlay

This demonstrates that processor is an instance of riscvonpynq.Processor.MixedProcessor. As we stated above, this means that the processor is connected to BRAM and DDR. riscvonpynq.Processor.MixedProcessor is an indirect subclass of pynq.overlay.DefaultHierarchy -- this means that the processor is actually a collection of IP wrapped in an Block Diagram Editor IP Hierarchy that is recognized by pynq using the `checkhierarchy` method.

The MixedProcessor class provides methods to run, launch (run a program asynchronously), and land (stop an asynchronous program). You can see further documentation in the cell below: 

In [ ]:
# custom0 opcode = X0001011 = 0B
# custom1 opcode = X0101011 = 2Bpicorv32_wrapper.bit
X0101011

In [2]:
%%riscvasm test_asm overlay.processor

.global main

main:
    #lw a2, 4(a1) # Get *argv[1]
    #lw a3, 0(a2) # Get argv[1]
    addi a0, zero, 0xFFFFFFFF     
    #addi a1, zero, 0xFFFFFFFF     
    #addi a2, zero, 0xFFFFFFFF 
    .insn r 0x2B, 0, 0, a0, a1, a2
    #add zero, zero, zero
    #add zero, zero, zero
    #add zero, zero, zero
    #add zero, zero, zero
    #.insn r 0x2B, 0, 0, a0, a1, a2
    ret

INIT


In [ ]:
    #addi a0, a3, -40 # Add -42, store in a0 (return register)


In [ ]:
opt/riscv32im/bin/riscv32-unknown-elf-objdump -d test_asm.o

In [4]:
test_asm.o:     file format elf32-littleriscv


Disassembly of section .text:

00000000 <main>:
   0:	0045a603          	lw	a2,4(a1)
   4:	00062683          	lw	a3,0(a2)
   8:	00c5850b          	0xc5850b
   c:	00c5852b          	0xc5852b
  10:	fd568513          	addi	a0,a3,-40
  14:	00008067          	ret

SyntaxError: invalid syntax (<ipython-input-4-7224ff00143d>, line 1)

In [3]:
import numpy as np
test_asm_arg = np.array([42], np.int32)
retval = overlay.processor.run(test_asm, test_asm_arg)
print(retval)
#if(retval != test_asm_arg[0] + (-42)):
#    print('Test failed!')
#else:
#    print('Test passed!')

-1


In [4]:
arr = overlay.processor.psBramController.mmio.array
for i in range(128):  
    print(f'Memory Index {i:3}: {arr[i]:#0{10}x}')

Memory Index   0: 0x00000013
Memory Index   1: 0x00000093
Memory Index   2: 0x00010137
Memory Index   3: 0x00000193
Memory Index   4: 0x00000213
Memory Index   5: 0x00000293
Memory Index   6: 0x00000313
Memory Index   7: 0x00000393
Memory Index   8: 0x00000413
Memory Index   9: 0x00000493
Memory Index  10: 0xffc12503
Memory Index  11: 0xff812583
Memory Index  12: 0x00000613
Memory Index  13: 0x00000693
Memory Index  14: 0x00000713
Memory Index  15: 0x00000793
Memory Index  16: 0x00000813
Memory Index  17: 0x00000893
Memory Index  18: 0x00000913
Memory Index  19: 0x00000993
Memory Index  20: 0x00000a13
Memory Index  21: 0x00000a93
Memory Index  22: 0x00000b13
Memory Index  23: 0x00000b93
Memory Index  24: 0x00000c13
Memory Index  25: 0x00000c93
Memory Index  26: 0x00000d13
Memory Index  27: 0x00000d93
Memory Index  28: 0x00000e13
Memory Index  29: 0x00000e93
Memory Index  30: 0x00000f13
Memory Index  31: 0x00000f93
Memory Index  32: 0x030000ef
Memory Index  33: 0xfeb12c23
Memory Index  